In [6]:
import os
import numpy as np 
import pandas as pd 

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score, confusion_matrix

import nltk
from nltk.classify import SklearnClassifier

import matplotlib.pyplot as plt
'exec(%matplotlib inline)'
from subprocess import check_output

In [7]:
data = pd.read_csv(os.getcwd()+"/input/Sentiment_GOP.csv", encoding = "ISO-8859-1")
data = data[['text','sentiment']]
data = data[data.sentiment!="Neutral"]

X_data = data['text']
Y_data = data['sentiment']

In [8]:
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords

stopwords_set = set(stopwords.words("english")) 
tknzr = TweetTokenizer()
stemmer = SnowballStemmer("english")


In [9]:
X_clean = []
for txt in X_data:
    words = [word.lower() for word in tknzr.tokenize(txt)] 
    words_cleaned = [word for word in words
        if 'http' not in word
        and not word.startswith('@')
        and not word.startswith('#')
        and word != 'rt']  
    words_wo_stop = [word for word in words_cleaned if not word in stopwords_set]  
#     tags = []
#     for word in words_wo_stop:
#         tags.extend(nltk.pos_tag([word]))
#     tags_root = [(stemmer.stem(tag[0]), tag[1]) for tag in tags]
#     words_root = [stemmer.stem(word) for word in words]
    txt_clean = ""
    for word in words_wo_stop:
        txt_clean += (word + " ")
    X_clean.append(txt_clean)

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer 
 
# settings that you use for count vectorizer will go here
tfidf_vectorizer=TfidfVectorizer(use_idf=True)
 
# just send in all your docs here
tfidf_vectorizer_vectors=tfidf_vectorizer.fit_transform(X_clean)

In [11]:
first_vector_tfidfvectorizer=tfidf_vectorizer_vectors[0]
 
# place tf-idf values in a pandas data frame
df = pd.DataFrame(first_vector_tfidfvectorizer.T.todense(), index=tfidf_vectorizer.get_feature_names(), columns=["tfidf"])
df.sort_values(by=["tfidf"],ascending=False)

,tfidf
catch,0.425320
lines,0.405005
seconds,0.390591
90,0.386581
full,0.351853
scott,0.290766
best,0.264719
last,0.193516
night,0.192867
pre,0.000000


In [44]:
# Extracting word features

#returns list of all words
def all_words(tweets):
    all = []
    for (tags, sentiment) in tweets:
        for tag in tags:
            all.append(tag[0])
    return all 

def all_words_tag(tweets):
    all = []
    for (tags, sentiment) in tweets:
        all.extend(tags)
    return all 


def freqs(wordlist):
    wordlist = nltk.FreqDist(wordlist)   
    #FreqDist is a class that is like dictionary with word as key and # of times it appears as value
    return wordlist

allW = all_words(tweets)
# allT= all_words_tag(tweets)
allFreqs = freqs(allW) #All unique words in all tweets
# allFreqsT = freqs(allT)
allKeys = allFreqs.keys()
# allKeysT = allFreqsT.keys()

def extract_features(document):
    docWords = [tag for (tag, sentiment) in document[0]]
#     docTags = [tag for tag in tags for (tags, sentiment) in tweets]
    docFreqs = nltk.FreqDist(docWords)
#     docFreqsT = nltk.FreqDist(docTags)
    docKeys = docFreqs.keys()
#     docKeysT = docFreqsT.keys()
    features = {}
    for allKey in allKeys:
        features[allKey] = (allKey in docKeys)
#         /allFreqs[allKey]          
    return features


In [45]:
print(tweets[0])

feats = extract_features(tweets[0])

print(feats)

([('sooooo', 'NN'), ('...', ':'), ('want', 'NN'), ('candid', 'NNS'), ('next', 'JJ'), ('presid', 'NN')], 'Negative')
{'sooooo': True, '...': True, 'want': True, 'candid': True, 'next': True, 'presid': True, 'part': False, 'debat': False, 'pander': False, 'god': False, 'endors': False, 'rand': False, 'paul': False, 'budget': False, 'multipl': False, 'set': False, 'book': False, 'yep': False, 'american': False, 'peopl': False, 'last': False, 'night': False, 'one': False, 'sens': False, 'engag': False, 'market': False, 'share': False, 'largest': False, 'ever': False, 'hey': False, 'let': False, 'digest': False, 'allow': False, 'crazi': False, 'trash': False, 'poll': False, 'twitter': False, 'follow': False, 'spike': False, 'saw': False, 'bunch': False, 'rich': False, 'old': False, 'despot': False, 'zero': False, 'solut': False, 'nation': False, 'problem': False, '11:11': False, 'pleas': False, 'better': False, 'futur': False, 'leader': False, 'countri': False, 'watch': False, 'fox': False,

In [46]:
# Training the Naive Bayes classifier
training_features = nltk.classify.apply_features(extract_features,tweets)
classifier = nltk.NaiveBayesClassifier.train(training_features)

ValueError: too many values to unpack (expected 2)

In [39]:
def accuracyScore(correct, total):
    return correct/total      #How likely that u answer right

def sensitivityScore(truePos, totalPos):
    return truePos/totalPos    #How likely detect positive when is positive
    
def specificityScore(trueNeg, totalNeg):
    return trueNeg/totalNeg   #How likely detect negative when is negative
    
def precisionScore(truePos, guessedPos):
    return truePos/guessedPos #How likely is positive when detect positive

def f1Score(sensitivity, precision):
    return 2*sensitivity*precision/(sensitivity+precision)  
    #weighted avg of recall and precision, useful if harm of falsePos and falseNeg differs


In [40]:
test_pred = [classifier.classify(extract_features(tknzr.tokenize(obj))) for obj in test_set]   

NameError: name 'classifier' is not defined

In [ ]:
cm = confusion_matrix(test_ans, test_pred, labels=["Positive", "Negative"])
tp, fn, fp, tn = cm.ravel()

In [ ]:
print(cm)

In [ ]:
print(tn, fp, fn, tp)

In [ ]:
print(accuracy_score(test_ans, test_pred))
print(recall_score(test_ans, test_pred, labels=["Positive", "Negative"], pos_label = "Positive"))
print(precision_score(test_ans, test_pred, labels=["Positive", "Negative"], pos_label = "Positive"))
print(f1_score(test_ans, test_pred, labels=["Positive", "Negative"], pos_label = "Positive"))

In [ ]:
accuracy = accuracyScore(tp+tn, tp+tn+fp+fn)
sensitivity = sensitivityScore(tp,tp+fn)
specificity = specificityScore(tn, tn+fp)
precision = precisionScore(tp, tp+fp)
f1 = f1Score(sensitivity, precision)

print(accuracy)
print(sensitivity)
print(specificity)
print(precision)
print(f1)